In [25]:

from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import mysql.connector
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, KFold, cross_val_predict, \
    cross_validate

with open('pss.txt') as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]
password = lines[0]
db = lines[1]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password=password,
    database=db,
    use_pure=True
)

my_cursor = mydb.cursor()
steps_title_list = []
d_steps_title = {}
university_name_list = []
d_uni_name = {}

def convert_nans(df):
    # job_applicant_id,gender,age,marriage_status,language,degree,skill,num_prev_company,work_interval,steps_title
    df['steps_title'] = df['steps_title'].fillna(6)
    df['age'] = df['age'].fillna(-1)
    df['skill'] = df['skill'].fillna(-1)
    df['marriage_status'] = df['marriage_status'].fillna(-1)
    # df['language'] = df['language'].fillna(-1)
    # df['job_contract_type'] = df['job_contract_type'].fillna(-1)
    df['gender'] = df['gender'].fillna(-1)
    df['degree'] = df['degree'].fillna(-1)
    df['skill'] = df['skill'].fillna(-1)
    df['num_prev_company'] = df['num_prev_company'].fillna(-1)
    df['work_interval'] = df['work_interval'].fillna(-1)





my_cursor.execute('select distinct steps_title from useful_data')
for item in my_cursor:
    steps_title_list.append(item)


for title in steps_title_list:
    if title[0] == 'نیازمند تعیین وضعیت':
        d_steps_title[title[0]] = 0
    elif title[0] == 'تایید برای مصاحبه':
        d_steps_title[title[0]] = 1
    elif title[0] == 'استخدام شده':
        d_steps_title[title[0]] = 2
    elif title[0] == 'رد شده':
        d_steps_title[title[0]] = 3
    elif title[0] == 'انصراف از مصاحبه':
        d_steps_title[title[0]] = 4
    else:
        d_steps_title[title[0]] = 6


df_train = pd.read_csv("train.csv")
df_train.head(10)
df_test = pd.read_csv("test.csv")
df_train['steps_title'] = df_train['steps_title'].map(d_steps_title)
df_test['steps_title'] = df_test['steps_title'].map(d_steps_title)
convert_nans(df_train)
convert_nans(df_test)
x_train =df_train[df_train.columns[1:9]]
x_test = df_test[df_test.columns[1:9]]
y_test = df_test['steps_title']
y_train = df_train['steps_title']
kfold = KFold(n_splits=10)
features_train = df_train.columns[1:9]
clf = RandomForestClassifier(n_jobs=2 , random_state=0)
cv_results = cross_val_score(clf,x_train , y_train,cv=kfold , scoring='accuracy')
output = cross_validate(clf, x_train, y_train, cv=2, scoring = 'accuracy', return_estimator =True)
y_pred=cross_val_predict(clf,x_test,y_test,cv=10)
clf.fit(x_train , y_train)
results = (cv_results.mean(),cv_results.std())
print(results)
print("accuracy : ", accuracy_score(y_test, y_pred))
pd.crosstab(y_test,y_pred)
for idx,estimator in enumerate(output['estimator']):
    print("Features sorted by their score for estimator {}:".format(idx))
    feature_importances = pd.DataFrame(estimator.feature_importances_,
                                        index= features_train,
                                        columns=['importance']).sort_values('importance', ascending=False)
    print(feature_importances)

(0.9733722322401567, 0.003765085322374464)
accuracy :  0.675531914893617
Features sorted by their score for estimator 0:
                 importance
age                0.344615
contract_type      0.216327
average_gpa        0.142243
languages          0.113096
degree             0.078479
skill              0.054479
gender             0.050761
marriage_status    0.000000
Features sorted by their score for estimator 1:
                 importance
age                0.350102
contract_type      0.210639
languages          0.135859
average_gpa        0.099385
degree             0.078982
skill              0.076215
gender             0.048817
marriage_status    0.000000
